In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
import numpy as np
import os
import utils
import random as rn

In [2]:
SEED = 123
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_seed(SEED)
np.random.seed(SEED)
rn.seed(SEED)

data = np.load(os.path.join('..','data','royalty.npz'))

entities = data['entities'].tolist()
relations = data['relations'].tolist()

NUM_ENTITIES = len(entities)
NUM_RELATIONS = len(relations)
EMBEDDING_DIM = 30
OUTPUT_DIM = 50
LEARNING_RATE = 1e-3
NUM_EPOCHS = 1
BATCH_SIZE = 32

ent2idx = dict(zip(entities, range(NUM_ENTITIES)))
rel2idx = dict(zip(relations, range(NUM_RELATIONS)))

triples, traces = data['grandmother_triples'], data['grandmother_traces']

train2idx = utils.array2idx(triples, ent2idx,rel2idx)

adj_mats = utils.get_adjacency_matrix_list(
    num_relations=NUM_RELATIONS,
    num_entities=NUM_ENTITIES,
    data=train2idx
)

train2idx = np.expand_dims(train2idx,axis=0)

all_indices = np.arange(NUM_ENTITIES).reshape(1,-1)

In [ ]:
class RGCN_Layer(tf.keras.layers.Layer):
    def __init__(self,num_relations,output_dim,**kwargs):
        super(RGCN_Layer,self).__init__(**kwargs)
        self.num_relations = num_relations
        self.output_dim = output_dim
        
    def build(self,input_shape):

        input_dim = int(input_shape[3][-1])
        
        self.relation_kernel = self.add_weight(
            shape=(self.num_relations,input_dim, self.output_dim),
            name="relation_kernels",
            trainable=True,
            initializer=tf.keras.initializers.RandomNormal(
                mean=0.0,
                stddev=1,
                seed=SEED
            )
        )


        self.self_kernel = self.add_weight(
            shape=(input_dim, self.output_dim),
            name="self_kernel",
            trainable=True,
            initializer=tf.keras.initializers.RandomNormal(
                mean=0.0,
                stddev=1,
                seed=SEED
            )
        )
    
    def call(self, inputs):
        
        embeddings,head_idx,tail_idx,head_e,tail_e,adj_mats = inputs
#         print('embeddings',embeddings.shape)
#         print('adj_mats',adj_mats.shape)
#         print('head_idx',head_idx.shape)
#         print('head_e',head_e.shape)
        adj_mats = tf.squeeze(adj_mats,axis=0)
        embeddings = tf.squeeze(embeddings,axis=0)
        print('head_e',head_e.shape)
        print('embeddings',embeddings.shape)
        print('head_idx',head_idx.shape)
        print('adj_mats',adj_mats.shape)

        head_output = tf.matmul(head_e,self.self_kernel)
        tail_output = tf.matmul(tail_e,self.self_kernel)
        print('head_output',head_output.shape)
        
        for i in range(self.num_relations):
            
            adj_i = adj_mats[i]
            print('adj_i',adj_i.shape)
            head_adj = tf.nn.embedding_lookup(adj_i,head_idx)
            print('head_adj',head_adj.shape)
            tail_adj = tf.nn.embedding_lookup(adj_i,tail_idx)
            
            h_head = tf.matmul(head_adj,embeddings)
            h_tail = tf.matmul(head_adj,embeddings)
            print('h_head',h_head.shape)
            head_output += tf.matmul(h_head,self.relation_kernel[i])
            tail_output += tf.matmul(h_tail,self.relation_kernel[i])

        print('head_output',head_output.shape)

        return head_output,tail_output

class DistMult(tf.keras.layers.Layer):
    def __init__(self, num_relations,**kwargs):
        super(DistMult,self).__init__(**kwargs)
        self.num_relations = num_relations
        
    def build(self,input_shape):
        
        embedding_dim = input_shape[0][-1]
        
        self.kernel = self.add_weight(
            shape=(self.num_relations,embedding_dim),
            trainable=True,
            initializer=tf.keras.initializers.RandomNormal(
                mean=0.0,
                stddev=1,
                seed=SEED
            ),
            name='rel_embedding'
        )
        
    def call(self,inputs):
        
        head_e,rel_idx,tail_e = inputs
        #print('dist rel_idx',rel_idx)
        rel_e = tf.nn.embedding_lookup(self.kernel,rel_idx)
        #print('dist rel_e',rel_e)
        return tf.sigmoid(tf.reduce_sum(head_e*rel_e*tail_e, axis=-1))
    
def get_RGCN_Model(num_entities,num_relations,embedding_dim,output_dim,seed):

    head_input = tf.keras.Input(shape=(None,), name='head_input',dtype=tf.int64)
    rel_input = tf.keras.Input(shape=(None,), name='rel_input',dtype=tf.int64)
    tail_input = tf.keras.Input(shape=(None,), name='tail_input',dtype=tf.int64)
    all_entities = tf.keras.Input(shape=(num_entities), name='all_entities',dtype=tf.int64)

    adj_inputs = tf.keras.Input(
        shape=(
            num_relations,
            num_entities,
            num_entities
        ),
        dtype=tf.float32,
        name='adj_inputs'
    )

    entity_embeddings = Embedding(
        input_dim=num_entities,
        output_dim=embedding_dim,
        name='entity_embeddings',
        embeddings_initializer=tf.keras.initializers.RandomUniform(
            minval=-1,
            maxval=1,
            seed=seed
        )
    )

    head_e = entity_embeddings(head_input)
    tail_e = entity_embeddings(tail_input)
    all_e = entity_embeddings(all_entities)

    new_head,new_tail = RGCN_Layer(num_relations=num_relations,output_dim=output_dim)([
        all_e,
        head_input,
        tail_input,
        head_e,
        tail_e,
        adj_inputs
        ]
    )

    output = DistMult(num_relations=num_relations,name='output')([
        new_head,rel_input,new_tail
        ]
    )

    model = tf.keras.Model(
        inputs=[
            all_entities,
            head_input,
            rel_input,
            tail_input,
            adj_inputs
        ],
        outputs=[
            output
        ]
    )

    return model

In [ ]:
model = get_RGCN_Model(
    num_entities=NUM_ENTITIES,
    num_relations=NUM_RELATIONS,
    embedding_dim=EMBEDDING_DIM,
    output_dim=OUTPUT_DIM,
    seed=SEED
)

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
bce = tf.keras.losses.BinaryCrossentropy()

data = tf.data.Dataset.from_tensor_slices((
        train2idx[0,:,0],
        train2idx[0,:,1],
        train2idx[0,:,2], 
        np.ones(train2idx.shape[1])
    )
).batch(BATCH_SIZE)

for epoch in range(NUM_EPOCHS):

    for pos_head,rel,pos_tail,y in data:

        neg_head, neg_tail = utils.get_negative_triples(
            head=pos_head, 
            rel=rel, 
            tail=pos_tail,
            num_entities=NUM_ENTITIES
        )

        with tf.GradientTape() as tape:
            
#             print(all_indices.shape)
#             print(pos_head.shape)
#             print(adj_mats.shape)

            y_pos_pred = model([
                all_indices,
                pos_head,
                rel,
                pos_tail,
                adj_mats
                ],
                training=True
            )

            y_neg_pred = model([
                all_indices,
                neg_head,
                rel,
                neg_tail,
                adj_mats
                ],
                training=True
            )

            y_pred = tf.concat([y_pos_pred,y_neg_pred],axis=0)
            y_true = tf.concat([y,tf.zeros_like(y)],axis=0)

            loss = bce(y_true,y_pred)

        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

    print(f'loss {loss} after epoch {epoch}')

In [ ]:
print(all_indices.shape)
print(train2idx[:,:,0].shape)
print(train2idx[:,:,1].shape)
print(train2idx[:,:,2].shape)
print(adj_mats.shape)

In [ ]:
preds = model.predict(
    x=[
        all_indices,
        train2idx[:,:,0],
        train2idx[:,:,1],
        train2idx[:,:,2],
        adj_mats
    ]
)

In [ ]:
preds.shape

In [3]:
class New_RGCN_Layer(tf.keras.layers.Layer):
    def __init__(self,num_relations,output_dim,**kwargs):
        super(New_RGCN_Layer,self).__init__(**kwargs)
        self.num_relations = num_relations
        self.output_dim = output_dim
        
    def build(self,input_shape):

        input_dim = int(input_shape[-2][-1])
        
        self.relation_kernel = self.add_weight(
            shape=(self.num_relations,input_dim, self.output_dim),
            name="relation_kernels",
            trainable=True,
            initializer=tf.keras.initializers.RandomNormal(
                mean=0.0,
                stddev=1,
                seed=SEED
            )
        )


        self.self_kernel = self.add_weight(
            shape=(input_dim, self.output_dim),
            name="self_kernel",
            trainable=True,
            initializer=tf.keras.initializers.RandomNormal(
                mean=0.0,
                stddev=1,
                seed=SEED
            )
        )
    
    def call(self, inputs):
        
        embeddings,head_idx,head_e,tail_idx,tail_e,adj_mats = inputs
        
#         print('embeddings',embeddings.shape)
#         print('head_idx',head_idx.shape)
#         print('head_e',head_e.shape)
#         print('adj_mats',adj_mats.shape)
            
        head_output = tf.matmul(head_e,self.self_kernel)
        tail_output = tf.matmul(tail_e,self.self_kernel)
        
        #print('head_output',head_output.shape)
        
        for i in range(self.num_relations):
            
            adj_i = adj_mats[i]

            #print('adj_i',adj_i.shape)
            
            head_adj = tf.nn.embedding_lookup(adj_i,head_idx)
            tail_adj = tf.nn.embedding_lookup(adj_i,tail_idx)
            
            #print('head_adj',head_adj.shape)
            
            #print('head_adj',head_adj.shape)
            #print('embeddings',embeddings.shape)
            
            head_update = tf.matmul(head_adj,embeddings)
            tail_update = tf.matmul(tail_adj,embeddings)

            head_output += tf.matmul(head_update,self.relation_kernel[i])
            tail_output += tf.matmul(tail_update,self.relation_kernel[i])
       
        return head_output, tail_output
    
class DistMult(tf.keras.layers.Layer):
    def __init__(self, num_relations,**kwargs):
        super(DistMult,self).__init__(**kwargs)
        self.num_relations = num_relations
        
    def build(self,input_shape):
        
        embedding_dim = input_shape[0][-1]
        
        self.kernel = self.add_weight(
            shape=(self.num_relations,embedding_dim),
            trainable=True,
            initializer=tf.keras.initializers.RandomNormal(
                mean=0.0,
                stddev=1,
                seed=SEED
            ),
            name='rel_embedding'
        )
        
    def call(self,inputs):
        
        head_e,rel_idx,tail_e = inputs
        
        rel_e = tf.nn.embedding_lookup(self.kernel,rel_idx)
        
        score = tf.sigmoid(tf.reduce_sum(head_e*rel_e*tail_e,axis=-1))
        
        return tf.expand_dims(score,axis=0)
#         embeddings,head_idx,tail_idx,head_e,tail_e,adj_mats = inputs

#         adj_mats = tf.squeeze(adj_mats,axis=0)
#         embeddings = tf.squeeze(embeddings,axis=0)

#         head_output = tf.matmul(head_e,self.self_kernel)
#         tail_output = tf.matmul(tail_e,self.self_kernel)
        
#         for i in range(self.num_relations):
            
#             adj_i = adj_mats[i]

#             head_adj = tf.nn.embedding_lookup(adj_i,head_idx)
#             tail_adj = tf.nn.embedding_lookup(adj_i,tail_idx)
            
#             h_head = tf.matmul(head_adj,embeddings)
#             h_tail = tf.matmul(head_adj,embeddings)
            
#             head_output += tf.matmul(h_head,self.relation_kernel[i])
#             tail_output += tf.matmul(h_tail,self.relation_kernel[i])

#         return head_output,tail_output

In [4]:
data = tf.data.Dataset.from_tensor_slices((
        train2idx[:,:,0],
        train2idx[:,:,1],
        train2idx[:,:,2], 
        np.ones(train2idx.shape[1]).reshape(1,-1)
    )
).batch(1)

for h,r,t,y in data:
    print(h)
    break

tf.Tensor([[ 583 5732  781 ... 1063 2747  816]], shape=(1, 4024), dtype=int64)


In [5]:
all_entities = tf.keras.Input(shape=(NUM_ENTITIES,), name='all_entities',dtype=tf.int64)
head_input = tf.keras.Input(shape=(None,), name='head_input',dtype=tf.int64)
rel_input = tf.keras.Input(shape=(None,), name='rel_input',dtype=tf.int64)
tail_input = tf.keras.Input(shape=(None,), name='tail_input',dtype=tf.int64)

adj_inputs = tf.keras.Input(
        shape=(
            NUM_RELATIONS,
            NUM_ENTITIES,
            NUM_ENTITIES
        ),
        dtype=tf.float32,
        name='adj_inputs'
    )

entity_embeddings = Embedding(
        input_dim=NUM_ENTITIES,
        output_dim=EMBEDDING_DIM,
        name='entity_embeddings',
        embeddings_initializer=tf.keras.initializers.RandomUniform(
            minval=-1,
            maxval=1,
            seed=SEED
        )
    )

all_e = entity_embeddings(all_entities)
head_e = entity_embeddings(head_input)
tail_e = entity_embeddings(tail_input)

all_e = tf.keras.layers.Lambda(lambda x:x[0,:,:])(all_e)
head_e = tf.keras.layers.Lambda(lambda x:x[0,:,:])(head_e)
tail_e = tf.keras.layers.Lambda(lambda x:x[0,:,:])(tail_e)

head_index = tf.keras.layers.Lambda(lambda x:x[0,:])(head_input)
rel_index = tf.keras.layers.Lambda(lambda x:x[0,:])(rel_input)
tail_index = tf.keras.layers.Lambda(lambda x:x[0,:])(tail_input)

adj_mats_layer = tf.keras.layers.Lambda(lambda x:x[0,:,:])(adj_inputs)
#embeddings,head_idx,head_e,tail_idx,tail_e,adj_mats

new_head,new_tail = New_RGCN_Layer(NUM_RELATIONS,OUTPUT_DIM)([all_e,head_index,head_e,tail_index,tail_e,adj_mats_layer])
#new_head = New_RGCN_Layer(NUM_RELATIONS,OUTPUT_DIM)([all_entities,])

output = DistMult(num_relations=NUM_RELATIONS,name='output')([new_head,rel_index,new_tail])

#output = tf.keras.layers.Reshape((1,))(output)
# m = tf.keras.Model([head_input],[out])
# m([train2idx[:,0:32,0]])
m = tf.keras.Model([all_entities,head_input,rel_input,tail_input,adj_inputs],[output])

m([np.arange(NUM_ENTITIES).reshape(1,-1),train2idx[:,0:32,0],train2idx[:,0:32,1],train2idx[:,0:32,2],adj_mats])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity t

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1.,
        1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.]],
      dtype=float32)>

In [6]:
m.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), 
    optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3)
)

#m.summary()
m.fit(x=[
    all_indices,
    train2idx[:,:,0],
    train2idx[:,:,1],
    train2idx[:,:,2],
    adj_mats
],
    y=np.ones(train2idx.shape[1]).reshape(1,-1),
    epochs=NUM_EPOCHS,
    batch_size=1,
    verbose=1
)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 916us/step - loss: 7.5171


In [7]:
preds = m.predict(
    x=[
        all_indices,
        train2idx[:,:,0],
        train2idx[:,:,1],
        train2idx[:,:,2],
        adj_mats
    ]
)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [8]:
preds.shape

(1, 4024)